In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-01-23_11:22
E:\RIS\Staff Folders\Samuel\Requests\SH\Safety_Data_Viewer


In [2]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')



Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-23 11:22:38


In [3]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-23 11:22:40


### Initial Table Containing All NYSDOT Information for Study Stretch. 

In [4]:
"""

--SIP 1319----------------------------------------------------------------------------------------------------------

DROP TABLE IF EXISTS sam_crashes_1319; 

CREATE TABLE sam_crashes_13191319 AS 

SELECT distinct nys_a.*
FROM archive."2019_11_13_nysdot_all" nys_a
JOIN archive."18d.2019-11-13_node" lion
on nys_a.nodeid::int = lion.nodeid::int
WHERE nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
and lion.nodeid::int in(SELECT nodeid 
                        FROM archive."18d.2019-11-13_node" 
                        WHERE masterid in (SELECT masterid
                                           FROM archive."18d.2019-11-13_node" 
                                           WHERE nodeid::int in (47444,100531,9031585,9031837)))
                        
GRANT ALL on sam_crashes_1319 to public;

"""

### Initial Table Containing All Fatality Information for Study Stretch. 

In [5]:
"""
--Fatals---------------------------

DROP TABLE IF EXISTS sam_fatalities_1319; 

CREATE TABLE sam_fatalities_1319 AS 

SELECT DISTINCT nodeid 
FROM archive."18d.2019-11-13_node" 
WHERE masterid in (SELECT masterid
                   FROM archive."18d.2019-11-13_node" 
                   WHERE nodeid::int in (47444,100531,9031585,9031837))

"""

### 1. Sip 1319
#### Created table in Pgsql containing all nysdot_all data for Sip: 1319 - 111TH ST to 108TH STT

In [4]:
sip_1319= cdb.dfquery("""SELECT * FROM sam_crashes_1319""")

### Injury Summary

In [5]:
inj_sum = cdb.dfquery("""



WITH data as(
SELECT "Type", sum("Total Injuries") "Total Injuries", sum("Severe Injuries") "Severe Injuries", sum("Fatalities") "Fatalities", sum("Severe Injuries"  + "Fatalities") as "KSI"
FROM (
    SELECT * FROM (
    --Injuries/Severity By Mode
    SELECT  CASE WHEN accd_type_int = 1 then '1. ped'
                 WHEN accd_type_int = 2 then '2. bicycle'
                 WHEN accd_type_int = 3 then '3. mvo' end as "Type"
        ,sum(num_of_inj) as "Total Injuries"
        ,sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as "Severe Injuries"
        ,0 as "Fatalities"
    FROM sam_crashes_1319
    GROUP BY  accd_type_int
    ORDER BY  accd_type_int
    )injuries

    UNION 

    SELECT * from (
    SELECT CASE WHEN pos = 'PD' then '1. ped'
                WHEN pos = 'BI' then '2. bicycle'
                WHEN pos in ('MO','PS','DR') then '3. mvo' end as "Type"
           ,0 as "Total Injuries"
           ,0 as "Severe Injuries"
           ,count(id_) as "Fatalities"
    FROM public.fatality_nycdot_current fat
    WHERE fat.nodeid::int in ( select * from sam_fatalities_1319)
    and date_part('year',acdate) between 2013 and 2017
    GROUP BY pos
    )fatalities

) inj_sum
GROUP BY "Type"
)


SELECT * from (
SELECT *
FROM data

UNION

SELECT 'Total' as "Type", "Total Injuries","Severe Injuries", "Fatalities", "KSI" 
FROM (select 'Total' as Year, 
      sum(data."Total Injuries") "Total Injuries",
      sum(data."Severe Injuries") "Severe Injuries",
      sum(data."Fatalities") "Fatalities",
      sum(data."KSI") "KSI"
      from data
      ) tot
)inj_summary
ORDER BY "Type"

""")

In [6]:
inj_sum

,Type,Total Injuries,Severe Injuries,Fatalities,KSI
0,1. ped,41,11,1,12
1,2. bicycle,5,1,0,1
2,3. mvo,159,12,0,12
3,Total,205,24,1,25


### Fatalities by Mode

In [10]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-22 12:33:39


In [7]:
fat = cdb.dfquery("""

--Fatalities by mode-------------------------------------------------------------

SELECT CASE WHEN pos = 'PD' then 'ped'
            WHEN pos = 'BI' then 'bicycle'
            WHEN pos in ('MO','PS','DR') then 'mvo' end as "Type"
      ,count(case when date_part('year',acdate) between 2013 and 2017 then id_ end) as "2013-2017 (5 Years)"
      ,count(case when date_part('year',acdate)>=2013 and acdate < '07-07-2019'::date then id_ end) as "2013-07/07/2019 (7 Years)"
FROM public.fatality_nycdot_current fat
WHERE  fat.nodeid::int in (select nodeid from sam_fatalities_1319)
GROUP BY pos


""") 


In [8]:
fat

,Type,2013-2017 (5 Years),2013-07/07/2019 (7 Years)
0,ped,1,1
1,bicycle,0,0


### Non-Fatal Injuries By Severity 

In [9]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-23 11:23:30


In [14]:
non_fat_sev = cdb.dfquery("""

--NON-FATAL INJURIES BY SEVERITY---------------------------------------------------


WITH data as(
SELECT * 
FROM (  SELECT 
        '1. A' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))) as "Total"
        FROM sam_crashes_1319

        union

        SELECT 
        '2. B' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'B'::text, ''::text))) as "Total"
        FROM sam_crashes_1319

        union

        SELECT 
        '3. C' as Severity,
        sum(CASE WHEN accd_type_int = 1 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Pedestrian,
        sum(CASE WHEN accd_type_int = 2 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) Bicyclist,
        sum(CASE WHEN accd_type_int = 3 then length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text)) end) "Motor Vehicle",
        sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'C'::text, ''::text))) as "Total"
        FROM sam_crashes_1319

        union


        SELECT
        '4. Unknown' as Severity,
        sum(case when accd_type_int = 1 and coalesce(length(ext_of_inj::text),0) != num_of_inj then (num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0) end) Pedestrian,
        sum(case when accd_type_int = 2 and coalesce(length(ext_of_inj::text),0) != num_of_inj then (num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0)  end) Bicyclist,
        sum(case when accd_type_int = 3 and coalesce(length(ext_of_inj::text),0) != num_of_inj then (num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0)  end) "Motor Vehicle",
        sum((num_of_inj + num_of_fat) - coalesce(length(ext_of_inj::text),0)) as "Total"
        FROM sam_crashes_1319

        )sev
        ORDER BY severity
)


SELECT * 
FROM ( SELECT * 
       FROM data

       UNION 

       SELECT '5. Total' as Severity, Pedestrian, Bicyclist, "Motor Vehicle", "Total"
       FROM (SELECT 'Total' as Year
            ,sum(data.Pedestrian) Pedestrian
            ,sum(data.Bicyclist) Bicyclist
            ,sum(data."Motor Vehicle") "Motor Vehicle"
            ,sum(data."Total") "Total"
            FROM data
              ) tot
        )non_fat_inj
ORDER BY severity




""")

In [15]:
non_fat_sev.fillna(0)

,severity,pedestrian,bicyclist,Motor Vehicle,Total
0,1. A,11,1,12,24
1,2. B,7,1,8,16
2,3. C,23,3,135,161
3,4. Unknown,0,0,4,4
4,5. Total,41,5,159,205


### Injuries by Year

In [16]:
inj_year = cdb.dfquery("""


--Injuries by year---------------------------------------------------



WITH data as (
SELECT 
case_yr::text as Year
,coalesce(sum(CASE WHEN accd_type_int = 1 THEN num_of_inj end),0) "Pedestrian" 
,coalesce(sum(CASE WHEN accd_type_int = 2 THEN num_of_inj end),0) "Bicyclist"
,coalesce(sum(CASE WHEN accd_type_int = 3 THEN num_of_inj end),0) "Motor Vehicle"
FROM sam_crashes_1319
GROUP by case_yr
order by case_yr
)

SELECT * FROM ( SELECT data.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
                FROM data

                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total
                FROM (SELECT 'Total' as Year, 
                      sum(data."Pedestrian") "Pedestrian",
                      sum(data."Bicyclist") "Bicyclist",
                      sum(data."Motor Vehicle") "Motor Vehicle"
                      FROM data
                    ) tot
               )inj_year
ORDER BY year

""")


In [17]:
inj_year

,year,Pedestrian,Bicyclist,Motor Vehicle,total
0,2013,15,1,32,48
1,2014,7,1,20,28
2,2015,7,1,22,30
3,2016,2,1,42,45
4,2017,10,1,43,54
5,Total,41,5,159,205


### Injuries By Time of Day By Type

In [18]:
inj_tod = cdb.dfquery("""


--Injuries by time of day by type---------------------------------------------------


WITH data as (
SELECT 
CASE WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END as Time,
coalesce(sum(CASE WHEN accd_type_int = 1 THEN num_of_inj END),0) "Pedestrian", 
coalesce(sum(CASE WHEN accd_type_int = 2 THEN num_of_inj END),0) "Bicyclist",
coalesce(sum(CASE WHEN accd_type_int = 3 THEN num_of_inj END),0) "Motor Vehicle"
FROM sam_crashes_1319
GROUP by CASE WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END
ORDER BY case WHEN date_part('hour',accd_tme) = 0 and date_part('minute',accd_tme) = 0 THEN '9. Unknown'
     WHEN date_part('hour',accd_tme) between 0 and 2 THEN '1. 00:00-03:00' 
     WHEN date_part('hour',accd_tme) between 3 and 5 THEN '2. 03:00-06:00' 
     WHEN date_part('hour',accd_tme) between 6 and 8 THEN '3. 06:00-09:00' 
     WHEN date_part('hour',accd_tme) between 9 and 11 THEN '4. 09:00-12:00' 
     WHEN date_part('hour',accd_tme) between 12 and 14 THEN '5. 12:00-15:00' 
     WHEN date_part('hour',accd_tme) between 15 and 17 THEN '6. 15:00-18:00' 
     WHEN date_part('hour',accd_tme) between 18 and 20 THEN '7. 18:00-21:00' 
     WHEN date_part('hour',accd_tme) between 21 and 23 THEN '8. 21:00-24:00' END)

SELECT * FROM (SELECT data.*, 
                "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total,
                (round((("Pedestrian"+"Bicyclist"+"Motor Vehicle")::float/(SELECT sum(num_of_inj) FROM sam_crashes_1319)::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE time != '9. Unknown'

                UNION 


                SELECT * , "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data.time != '8. Unknown/Indeterminate'
                and time = '9. Unknown'


                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist"+"Motor Vehicle" Total, 'N/A' as "Percent of Known Injuries"
                FROM (SELECT 'Total' as Time, 
                      sum(data."Pedestrian") "Pedestrian",
                      sum(data."Bicyclist") "Bicyclist",
                      sum(data."Motor Vehicle") "Motor Vehicle"
                      FROM data
                      ) tot
                )inj_tod
ORDER BY time


""")

In [19]:
inj_tod

,time,Pedestrian,Bicyclist,Motor Vehicle,total,Percent of Known Injuries
0,1. 00:00-03:00,0,0,11,11,5.4000
1,2. 03:00-06:00,3,0,3,6,2.9000
2,3. 06:00-09:00,6,1,11,18,8.8000
3,4. 09:00-12:00,3,1,33,37,18.0000
4,5. 12:00-15:00,6,1,26,33,16.1000
5,6. 15:00-18:00,9,2,27,38,18.5000
6,7. 18:00-21:00,2,0,33,35,17.1000
7,8. 21:00-24:00,12,0,14,26,12.7000
8,9. Unknown,0,0,1,1,N/A
9,Total,41,5,159,205,N/A


### Pedestrian Injuries By Control And Pedestrian Action

In [20]:
ped_inj_con = cdb.dfquery("""


--Pedestrian Injuries By Control and Pedestrian Action---------------------------------------------------


WITH data as (
SELECT 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
FROM sam_crashes_1319
WHERE accd_type_int = 1
GROUP by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
              WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
              WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
              WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
              WHEN PED_LOC = '2' THEN '5. Midblock'
              WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
              ELSE '8. Unknown/Indeterminate' END

)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = '8. Unknown/Indeterminate'

                UNION

                SELECT 
                '7. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) ped_inj_con
ORDER BY " "

""")

In [21]:
ped_inj_con

,,Number of Injuries,Percent of Known Injuries
0,1. Signalized Intersection: Crossing WITH Signal,17,48.6000
1,2. Signalized Intersection: Crossing Against S...,8,22.9000
2,4. Other Actions/Uncontrolled Intersection,4,11.4000
3,5. Midblock,6,17.1000
4,7. Total Known,35,100.0000
5,8. Unknown/Indeterminate,6,N/A
6,Total,41,N/A


### Bicycle Injuries By Intersection Control and Bicyclist Action

In [22]:
bi_inj_con = cdb.dfquery("""


--Bicycle Injuries By Intersection Control and Bicyclist Action---------------------------------------------------



WITH data as (
SELECT 
CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END as " ", 
coalesce(sum(num_of_inj),0) "Number of Injuries"
FROM sam_crashes_1319
WHERE accd_type_int = 2
GROUP by CASE WHEN TRAF_CNTL = '02' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '1. Signalized Intersection: Crossing WITH Signal'
     WHEN TRAF_CNTL = '02' AND PED_ACTN = '02' AND PED_LOC = '1' THEN '2. Signalized Intersection: Crossing Against Signal'
     WHEN TRAF_CNTL = '03' AND PED_LOC = '1' THEN '3. Stop-Controlled Intersection / Crosswalk'
     WHEN TRAF_CNTL = '01' AND PED_LOC = '1' THEN '4. Other Actions/Uncontrolled Intersection'
     WHEN PED_LOC = '2' THEN '5. Midblock'
     WHEN TRAF_CNTL = '20' AND PED_ACTN = '01' AND PED_LOC = '1' THEN '6. Other Control Types'
     ELSE '8. Unknown/Indeterminate' END

)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = '8. Unknown/Indeterminate'

                UNION

                SELECT 
                '7. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != '8. Unknown/Indeterminate')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != '8. Unknown/Indeterminate'

                UNION


                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) bi_inj_con
ORDER BY " " 

""")

In [23]:
bi_inj_con

,,Number of Injuries,Percent of Known Injuries
0,1. Signalized Intersection: Crossing WITH Signal,1,50.0000
1,2. Signalized Intersection: Crossing Against S...,1,50.0000
2,7. Total Known,2,100.0000
3,8. Unknown/Indeterminate,3,N/A
4,Total,5,N/A


In [24]:
mvo_inj_coll = cdb.dfquery("""


--Motor Vehicle Injuries By Collision Type---------------------------------------------------


WITH data as(
SELECT 
CASE WHEN collision_ in ('03','10') THEN '1. LEFT Turn'
     WHEN collision_ in ('05','06') THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     WHEN collision_ in ('00','ZZ') THEN 'Unknown' END as " ",
sum(num_of_inj) as "Number of Injuries"--, crashid
FROM sam_crashes_1319
WHERE accd_type_int = 3
GROUP by CASE WHEN collision_ in ('03','10') THEN '1. LEFT Turn'
     WHEN collision_ in ('05','06') THEN '2. Right Turn'
     WHEN collision_ = '01' THEN '3. Rear-End'
     WHEN collision_ = '04' THEN '4. Right-Angle'
     WHEN collision_ = '02' THEN '5. Sideswipe(Same Direction)'
     WHEN collision_ = '07' THEN '6. Head-on'
     WHEN collision_ = '08' THEN '7. Sideswipe(Opposite Direction)'
     WHEN collision_ = '09' THEN '8. Other Known'
     WHEN collision_ in ('00','ZZ') THEN 'Unknown' END--, crashid
order by " "
)

SELECT * FROM ( SELECT data.*, 
                (round((("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != 'Unknown'

                UNION

                SELECT data.*, 'N/A' as "Percent of Known Injuries"
                FROM data
                WHERE data." " = 'Unknown'

                UNION

                SELECT 
                '9. Total Known' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , (round((sum("Number of Injuries")/(SELECT sum("Number of Injuries") FROM data WHERE data." " != 'Unknown')::float)::numeric, 3)*100.0)::text as "Percent of Known Injuries"
                FROM data
                WHERE data." " != 'Unknown'

                UNION

                SELECT 
                'Total' as " " 
                , sum("Number of Injuries") as "Number of Injuries"
                , 'N/A' as "Percent of Known Injuries"
                FROM data
                ) ped_inj_con
ORDER BY " " 


"""
                          )

In [25]:
mvo_inj_coll

,,Number of Injuries,Percent of Known Injuries
0,1. LEFT Turn,13,10.3000
1,2. Right Turn,6,4.8000
2,3. Rear-End,21,16.7000
3,4. Right-Angle,35,27.8000
4,5. Sideswipe(Same Direction),20,15.9000
5,7. Sideswipe(Opposite Direction),0,0.0000
6,8. Other Known,31,24.6000
7,9. Total Known,126,100.0000
8,Total,159,N/A
9,Unknown,33,N/A


### Injuries By Age Group

In [26]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-23 11:28:22


In [27]:
inj_age = cdb.dfquery("""


--Injuries By Age GROUP---------------------------------------------------


WITH data as(
SELECT 
CASE WHEN ages.age::int between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
     WHEN ages.age::int between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
     WHEN ages.age::int between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
     WHEN ages.age::int between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
     WHEN ages.age is null or num_of_inj!=1 THEN '5. Unknown' End "Age GROUP",
coalesce(sum(case when accd_type_int = 1 then num_of_inj end),0) "Pedestrian", 
coalesce(sum(case when accd_type_int = 2 then num_of_inj end),0) "Bicyclist" 
FROM sam_crashes_1319
LEFT JOIN (SELECT nys_v.crashid, age 
           FROM (SELECT distinct crashid
                 FROM archive."2019_11_13_nysdot_vehicle"
                 WHERE age between 1 and 120
                 and veh_typ in ('5','6')
                 GROUP by crashid
                 HAVING count(crashid) =1
                 ) nys_v
           JOIN archive."2019_11_13_nysdot_vehicle" nys_v_age
           on nys_v.crashid = nys_v_age.crashid
           WHERE veh_typ in ('5','6')
          ) ages
on sam_crashes_1319.crashid = ages.crashid
GROUP by CASE WHEN ages.age::int between 1 and 17 and num_of_inj = 1 THEN '1. Children(1-17)'
              WHEN ages.age::int between 18 and 29 and num_of_inj = 1 THEN '2. Young Adults(18-29)'
              WHEN ages.age::int between 30 and 64 and num_of_inj = 1 THEN '3. Adults(30-64)'
              WHEN ages.age::int between 65 and 120 and num_of_inj = 1 THEN '4. Seniors(65-120)'
              WHEN ages.age is null or num_of_inj!=1 THEN '5. Unknown' End
ORDER BY "Age GROUP"
)

SELECT * FROM ( SELECT data.*, "Pedestrian"+"Bicyclist" Total
                FROM data

                UNION 

                SELECT tot.*, "Pedestrian"+"Bicyclist" Total
                FROM (SELECT 'Total' as Year 
                     ,sum(data."Pedestrian") "Pedestrian"
                     ,sum(data."Bicyclist") "Bicyclist"
                     FROM data
                      ) tot

               ) inj_age
                
ORDER BY "Age GROUP"

""")

In [28]:
inj_age

,Age GROUP,Pedestrian,Bicyclist,total
0,1. Children(1-17),4,1,5
1,2. Young Adults(18-29),10,2,12
2,3. Adults(30-64),12,2,14
3,4. Seniors(65-120),5,0,5
4,5. Unknown,10,0,10
5,Total,41,5,46


In [29]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-01-23 11:28:42


### Ped Injuries By Veh Action And Pre Action

In [30]:
ped_veh_actn = cdb.dfquery("""


WITH data as(
SELECT 
CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
     WHEN pre_accd_actn = '02' THEN '2. Right Turn'
     WHEN pre_accd_actn = '01' THEN '3. Going Straight'
     WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
     WHEN pre_accd_actn = '15' THEN '5. Backing'
     WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'   
     WHEN pre_accd_actn in ('??','YY','XX', 'ZZ') or single_veh.crashid is null THEN '7. Unknown' END as " ",
coalesce(sum(CASE WHEN ped_actn = '01' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing WITH Signal",
coalesce(sum(CASE WHEN ped_actn = '02' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing Against Signal",
coalesce(sum(CASE WHEN ped_actn = '03' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing No Signal Marked Crosswalk",
coalesce(sum(CASE WHEN ped_actn = '04' and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Crossing No Signal or Crosswalk",
coalesce(sum(CASE WHEN ped_actn not in ('01','02','03','04','??','YY','XX','ZZ') and num_of_inj = 1 and single_ped.crashid is not null THEN num_of_inj END),0) as "Others",
coalesce(sum(CASE WHEN ped_actn in ('??','YY','XX', 'ZZ') or single_ped.crashid is null or (num_of_inj>1 and single_ped.crashid is not null) THEN num_of_inj END),0) as "UnKnown"
FROM sam_crashes_1319
LEFT JOIN (SELECT distinct crashid
         FROM archive."2019_11_13_nysdot_vehicle"
         WHERE veh_typ = '6'
         GROUP by crashid
         HAVING count(crashid) = 1) single_ped
     on sam_crashes_1319.crashid = single_ped.crashid
LEFT JOIN (SELECT sv.crashid, sv2.pre_accd_actn
       FROM (SELECT distinct crashid--, pre_accd_actn
                 FROM archive."2019_11_13_nysdot_vehicle"
                 WHERE veh_typ not in ('5','6')		
                 GROUP by crashid--, pre_accd_actn
                 HAVING count(crashid) = 1)sv
         LEFT JOIN (SELECT distinct crashid, pre_accd_actn
               FROM archive."2019_11_13_nysdot_vehicle"
               WHERE veh_typ not in ('5','6')
               --WHERE pre_accd_actn not in ('??','YY','XX')
               ) sv2
            on sv.crashid = sv2.crashid) single_veh
    on sam_crashes_1319.crashid = single_veh.crashid
WHERE accd_type_int = 1
GROUP by CASE WHEN pre_accd_actn = '03' THEN '1. Left Turn'
          WHEN pre_accd_actn = '02' THEN '2. Right Turn'
              WHEN pre_accd_actn = '01' THEN '3. Going Straight'
          WHEN pre_accd_actn = '04' THEN '4. Making U Turn'
          WHEN pre_accd_actn = '15' THEN '5. Backing'
          WHEN pre_accd_actn not in ('01','02','03','04','15','??','YY','ZZ') THEN '6. Other'     
          WHEN pre_accd_actn in ('??','YY','XX', 'ZZ') or single_veh.crashid is null THEN '7. Unknown' END
)


SELECT * FROM ( SELECT data.*, "Crossing WITH Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
        FROM data

        UNION 

        SELECT tot.*, "Crossing WITH Signal" + "Crossing Against Signal" + "Crossing No Signal Marked Crosswalk" + "Crossing No Signal or Crosswalk" +  "Others" + "UnKnown" Total
        FROM (SELECT 'Total' as " ", 
              sum(data."Crossing WITH Signal") "Crossing WITH Signal",
              sum(data."Crossing Against Signal") "Crossing Against Signal",
              sum(data."Crossing No Signal Marked Crosswalk") "Crossing No Signal Marked Crosswalk",
              sum(data."Crossing No Signal or Crosswalk") "Crossing No Signal or Crosswalk",
              sum(data."Others") "Others",
              sum(data."UnKnown") "UnKnown"
              FROM data
            ) tot

        ) ped_veh_actn
ORDER BY " "


""")

In [31]:
ped_veh_actn

,,Crossing WITH Signal,Crossing Against Signal,Crossing No Signal Marked Crosswalk,Crossing No Signal or Crosswalk,Others,UnKnown,total
0,1. Left Turn,4,1,0,0,0,3,8
1,2. Right Turn,4,2,0,0,0,0,6
2,3. Going Straight,4,8,0,2,1,9,24
3,7. Unknown,1,1,0,0,0,1,3
4,Total,13,12,0,2,1,13,41
